## DESE61003 - Audio Experience Design
# Week 4 - Making Sound (using Python)

In [ ]:
import numpy as np
from scipy import signal
import ipywidgets as widgets
from disiple.signals import AudioSignal, Spectrum
from disiple.operations import apply_adsr, apply_gain

### 1. Creating Random Noise

Audio in digital form is usually represented as a list of floating point numbers between -1 and 1. This is mostly a scientific convention as all sorts of data types (e.g. integers or floating points) and ranges (determined by bith depth, e.g. 16 or 24 bit encoding) are used to store audio in practice, depending on the specific storage format that's being used, but that is a different topic. Nonetheless, following this convention means that any one-dimensional array of numbers in the range [-1, 1] can be interpreted as sound.

Therefore we can use standard scientific Python libraries like [NumPy](https://numpy.org/) to generate sound. Take for instance a random vector of length 10,000. Note that we're using the modern way to generate random numbers with NumPy, see the [guide](https://numpy.org/doc/stable/reference/random/index.html#quick-start) for more info.

In [ ]:
random_gen = np.random.default_rng()
random_samples = random_gen.uniform(-1, 1, 10000)
random_samples[:5]

All that's left to do is to decide what sample rate to use. When digitising analogue audio, this number indicates how often we sample the analogue waveform, but in our scenario, it can be interpreted as how quickly do we play back our random array (how many samples per second).

We are using the `disiple` library to make it easy to visualise and play back audio. It provides the `AudioSignal` data type (already loaded from the module `disiple.signals` in the first code cell at the top), which we instantiate with our samples and a sample rate, as follows:

In [ ]:
random_signal_8kHz = AudioSignal(random_samples, samplerate=8000)
type(random_signal_8kHz)

The result is a variable of type `AudioSignal`, which has some convenient functionality. First of all, we can use its `play()` method to play back the sound.

In [ ]:
random_signal_8kHz.play()

We can also use the `display()` method to show an interactive graphical representation of the waveform.

In [ ]:
random_signal_8kHz.display()

This method also gets called automatically when a `AudioSignal` variable is written on the last line of a notebook cell, like this:

In [ ]:
random_signal_8kHz

One advantage of calling `display()` explicitly, is that arguments can be passed to modify the default visualisation. We can for instance zoom to the first 30 ms of the signal, and check that the samples are indeed quite random.

In [ ]:
random_signal_8kHz.display(x_range=(0, 0.03), title='Random audio signal sampled at 8 kHz')

If you want to get deeper into customising plots later on, then it will be helpful to know that this functionality is built around the [Bokeh](http://bokeh.org/) library and that you can pass any of the [keyword arguments of the `Figure` class](https://docs.bokeh.org/en/2.4.0/docs/reference/plotting/figure.html#bokeh.plotting.Figure) onto the `display()` method. For now, it suffices to be aware of the most important ones, namely the `width` and `height` (in pixels) of the figure, `x_range` and `y_range` to set the axes, and `title`. They're all set to reasonable defaults, so you don't need to worry about them for now.

#### Activity 1

Create two other signals with higher sample rates from the same random samples. What differences can you hear and see? Remember that the [Nyquist-Shannon theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) determines the highest frequency that can be represented.

In [ ]:
# your code here

_your answer here_

### 2. Frequency Analysis

To get a better idea of what is going on in an audio signal, we can plot its magnitude spectrum (the absolute value of the complex spectrum calculated using the Fourier transform). There's another class in `disiple`, sensibly named `Spectrum`, that does this for you.

In [ ]:
Spectrum(random_signal_8kHz).display(title='Spectrum of a random audio signal sampled at 8 kHz')

As expected, we can see that every frequency is present in approximately equal amounts - bar a few outliers due to the randomness - in the entire frequency range between 0 and the Nyquist frequency (4000 Hz here, because of our 8000 Hz sample rate). Pay attention to the y-axis and note that the spectrum gets visualised in decibels (dB) by default. Decibels represent the power per frequency component (the square of the magnitude) and correlate better with the sensitivity of our ears, but also tend to overexpose the quiet (low magnitude) outliers. In general, it is safe to ignore the exact shape of the spectrum at the lowest dBs, since a small difference in magnitude will lead to a large difference in dB. Compare this plot with the one below, where the linear values of the magnitude are shown, which gives less prominence to the quiet outliers.

In [ ]:
Spectrum(random_signal_8kHz, dB=False).display(title='Linear spectrum of a random audio signal sampled at 8 kHz')

#### Activity 2

Plot the spectra for the two other signals you created from the same random samples. What can you tell about them? Note the difference in x-axis when displaying the entire spectrum. You can use the `x_range` argument in `display()` to set all plots to the same scale.

In [ ]:
# your code here

_your answer here_

### 3. The Sound Of Silence

After all this random noise, it's about time we get a bit more control over the sounds we're creating. Since a waveform shows the evolution of amplitude over time, we will no longer create some samples and interpret them as a time-varying signal, but we will do it the proper way around. So let's start by creating a time vector, our x-axis. We'll choose a sample rate, for instance 44.1 kHz, which is the standard for consumer audio (CDs, streaming services, etc.), and a duration for our signals.

In [ ]:
samplerate = 44100 # in Hz
duration = 1 # in seconds

We can then create a vector of discret points in time that are spaced apart according to the sample rate $f_s$, meaning each point is $T = \dfrac{1}{f_s}$ seconds apart. Do verify this below! The NumPy function [`np.arange`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) is designed to generate such equidistant points in an interval.

In [ ]:
time = np.arange(0, duration, 1/samplerate)
with np.printoptions(precision=6, suppress=True):
    print(time)

Our first signal will be an easy one. We'll set the amplitude at each point in time to zero. As you can imagine, this will create silence. We can either take the mathematical approach to creating the amplitudes by implementing the function $f(t) = 0$ as vector arithmetic:

In [ ]:
silence = AudioSignal(0*time, samplerate)

Or take the pragmatic approach and use NumPy's [`np.zeros()`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) function to create a new vector of zeros of the desired length.

In [ ]:
silence = AudioSignal(np.zeros(len(time)), samplerate)

The result in any case is a (not very interesting) silent signal, but it's good to verify that it looks and sounds like it should. Before you do, think for a second about how you'd expect the spectrum of silence to look like.

In [ ]:
silence.play()
silence.display(title='Waveform of silence')
Spectrum(silence).display(title='Spectrum of silence')

If you're very observant and notice (or remember) that the spectrum is plotted in dB, you might be somewhat surprised to see the spectrum of silence drawn as a flat line (so far so good) at -120 dB (?). If this looks somewhat arbitrary, that's because it is. As you might imagine, the spectrum of silence has a (linear) value of zero for all frequencies. If we'd convert that to dB though, the value would be $-\infty$, which is rather hard to plot. Therefore the dB range in this particular implementation is set to have an arbitrary lower limit of -120 dB, and anything below that gets clipped.

#### Activity 3

Continuing our tour of [array creating functionality in NumPy](https://numpy.org/doc/stable/user/basics.creation.html#general-ndarray-creation-functions), we also have [`np.ones()`](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) to create an array of only ones. Use this function to create a waveform containing nothing but ones, plot it and its spectrum, and play it back. What do you expect it to sound like?

If you're wondering about a more mathematical approach to create $f(t) = 1$ by computing a vector of all ones from the time vector, the [sign function](https://en.wikipedia.org/wiki/Sign_function) comes close (what is the difference?). It's also available in NumPy as [`np.sign`](https://numpy.org/doc/stable/reference/generated/numpy.sign.html).

<font size="10" style="float:left; margin: 10px">&#9888;</font>
<div style="color: white; background-color: indianred; padding: 10px; border: 1px solid red;">
Whenever you're listening to a sound you created, especially when wearing headphones, make sure you listen the first time with the volume down and if necessary, slowly increase it over multiple repetitions. Some waveforms are perceived louder than others, and it's easy to make an error and accidentally create a much louder sound than the previous one that you used to calibrate your volume with. It's always best to check your sound and its spectrum visually before listening to it.
</div>

In [ ]:
# your code here

_your answer here_

### 4. Creating Basic Waveforms

With the special cases out of the way, we can move on to the most common waveforms, starting with the sine wave. It is defined as $f(t) = sin(2\pi f t)$, where $f$ is the frequency of the waveform. Using [`np.sin()`](https://numpy.org/doc/stable/reference/generated/numpy.sin.html) on our time vector, its implementation becomes quite straightforward.

In [ ]:
freq = 220
sine_signal = AudioSignal(np.sin(2*np.pi*freq*time), samplerate)

We can verify that the signal indeed looks and sounds like a sine wave and that its spectrum contains a single component.

In [ ]:
sine_signal.play()
sine_signal.display(x_range=(0, 0.01), title='Waveform of a sine wave')
Spectrum(sine_signal).display(title='Spectrum of a sine wave')

To create the other basic waveforms, namely square, triangular and sawtooth waves, we're going to use functions from the [`signal`](https://docs.scipy.org/doc/scipy/reference/signal.html#waveforms) module from the [`scipy`](https://scipy.org/) library. This module has already been imported into our namespace as `signal`, so be careful to not overwrite it by naming one of your variables `signal`. If you still happen to do this, just execute the cell with the `import` statements at the top of this notebook again.


#### Activity 4

Read through the help for [`signal.square()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.square.html) (for square waves) and [`signal.sawtooth()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sawtooth.html) (both for triangular and sawtooth waves) to learn how to generate these waveforms. Then create signals with the same frequency as the sine wave, play them back and plot waveforms and spectra. See if you can create both sawtooth waves with rising and falling slopes. What do you notice about their spectra?

##### Square wave

In [ ]:
# your code here

##### Triangular wave

In [ ]:
# your code here

##### Rising sawtooth wave

In [ ]:
# your code here

##### Falling sawtooth wave

In [ ]:
# your code here

##### Discussion

_your answer here_

### 5. Adding Temporal Variation

Natural sounds evolve over time, unlike the perfectly periodic waveforms we've been generating so far. A natural signal typically has an impulsive start, takes some time to settle and finally fades out. This has been especially noticeable with the signal of ones, where the abrupt start and ending caused blips. We can approximate natural evolution over time by applying an _ADSR (attack-decay-sustain-release) curve_ as a so-called _envelope_ to the amplitude of a signal. 

An ADSR curve is defined by its four sections, which are determined by four parameters. During the attack phase, the amplitude rises from zero to maximum, then drops back to the sustain amplitude during the decay, stays constant for the sustain and then drops further to zero during the release. Consequently, we only need to specify the sustain amplitude as a parameter to determine the whole amplitude evolution over time. The other three parameters are the duration of the attack, decay and release phases. The sustain simply lasts for as long as needed to match the signal duration or in case of real-time processing, until a command to end the sustain phase is received, e.g. when a key is released.

The ADSR curve is one of the defining characteristics of a musical instrument, and is tied to the way the sound is produced. For instance, a sound produced by a flute starts relatively gentle (a smooth attack) because of the soft blow into the tube, stays at the same high level for as long as the air keeps flowing (nearly no decay, long sustain) and dissipates quickly afterwards (short release). A violin and other bowed instruments have an even longer attack and a similar long sustain (as long as the bow moves). Plucked string instruments, on the other hand, have a sharper (short) attack and decay coinciding with the string excitation followed by a very gradual release while the string keeps ringing. Finally, percussive instruments generally have no sustain at all, just a fast attack and decay.

A function `apply_adsr(signal, sustain_amp, attack, decay, release)` is included in the `disiple` library and already loaded. You need to specify `sustain_amp` as a value between 0 and 1 and the `attack`, `decay` and `release` times in seconds.

#### Activity 5

Experiment with applying ADSR curves with different parameters to the signals we've previously created. First think about what signal you can apply the ADSR curve to in order to best visualise the shape of the curve itself. Then try some other signals and verify the effect aurally and visually.

In [ ]:
# your code here

_your answer here_

Although ADSR curves as you would encouter them on synthesisers are indeed controlled just by the aforementioned four parameters, there actually is another source of variation between ADSR curves. The shape of the rising and falling amplitude slopes can also be considered a parameter. In the previous example these slopes were linear, but another popular choice is to make them exponential. Unlike the precisely defined linear slope, there are many variations that can all be considered exponential, and different manufacturers pick different variations, which contributes to the characteristics of a specific instrument.

You can pass the optional argument `shape`, with values between 0 and $\infty$ to `apply_adsr()` to try some alternative shapes. A value of `shape=1` (the default) will give you a linear slope. In order to get a quick idea of the influence of the shape parameter, you can play around with the graphical calculater embedded below, where the shape parameter is controlled by the variable `h`.

In [ ]:
from IPython.display import IFrame
IFrame(src='https://www.desmos.com/calculator/j600aw9eqc', width="1000", height="500")

#### Activity 6

Visualise an ADSR curve with an exponential shape and try changing the shape of some of your previously created ADSR curves. Are you able to hear any difference?

In [ ]:
# your code here

### 6. Additive Synthesis

Now that we know all there is to know about the basic waveforms, we can start combining them to create additional sounds. Additive synthesis simply means that multiple waveforms get summed together. Typically this is done with multiple sine waves, for reasons of simplicity. When multiple complex waveforms get added, their many harmonics will interact in ways that are not necessarily intuitive, making it conceptually harder to control the result and achieve the desired sound. Do try it if you want though.

As you'd expect, variables of type `AudioSignal` can be added together using the `+` operator, which returns a new signal, or using the `+=`, which adds to the first signal in-place. The only pitfall to be aware of when adding multiple waveforms, is that you need to make sure that maximum amplitude of the summed signal does not exceed 1. Otherwise the signal will no longer be a valid audio signal and will throw an error when you try to play it back. Because we've been creating waveforms that oscillate between -1 and 1 so far, the likelihood that the absolute value of their sum exceeds 1 is very high.

In [ ]:
sine_220Hz = AudioSignal(np.sin(2*np.pi*220*time), samplerate)
sine_440Hz = AudioSignal(np.sin(2*np.pi*440*time), samplerate)
sine_mix = sine_220Hz + sine_440Hz
sine_mix.display(x_range=(0, 0.03), y_range=(-2.09, 2.09))
try:
    sine_mix.play()
except ValueError:
    print('Playing back an audio signal with amplitude outside of [-1,1] throws a ValueError')

The solution is to scale down the amplitude of our sinusoidal components before adding them together. This can be done by scaling the samples that are used to create an `AudioSignal`, for instance by $0.5$.

In [ ]:
scaled_sine = AudioSignal(0.5 * np.sin(2*np.pi*freq*time), samplerate)
scaled_sine.play()
scaled_sine.display(title='A quieter sine wave')

As expected, the resulting signal sounds quieter, though not exactly half as loud because of the (approximately) logarithmic sensitivity of our ears to loudness.

It is also possible to change the amplitude once an `AudioSignal` has been created. The function `apply_gain(signal, amount, dB=True)` is already loaded from the `disiple` toolbox. By default, the amount of _gain_ (a signal processing term roughly meaning volume) is expressed in decibels, with the reference level of 0 dB equivalent to the maximum amplitude of 1. Reducing the amplitude therefore requires passing a negative amount. Alternatively, by setting the argument `dB=False` you can also pass values between 0 and 1 to scale the amplitude linearly.

In [ ]:
sine_mix_3dBless = apply_gain(sine_mix, -3)
sine_mix_3dBless.play()
sine_mix_3dBless.display(title='Sine mixture reduced by 3 dB')
sine_mix_half = apply_gain(sine_mix, 0.5, dB=False)
sine_mix_half.play()
sine_mix_half.display(title='Sine mixture with amplitude reduced to half')

A final piece of functionality included in the `apply_gain` function is that it clips all amplitudes that fall outside the allowed range. All values greater than 1 will be set to 1 and the same for values lower than -1. This ensure that the output of `apply_gain` always is a valid `AudioSignal`, regardless of the amount of gain. You could even use the function as a clipper only, since the default gain is 0 dB. Beware though, the process of hard limiting a signal (as clipping is also known as) is a harsh form of distortion, which strongly modifies the signal and introduces many additional harmonics. Therefore it is not an alternative to scaling down the signal, but rather an ensurance against invalid audio signals, or even an audio effect.

In [ ]:
clipped_sine_mix = apply_gain(sine_mix)
clipped_sine_mix.play()
clipped_sine_mix.display(x_range=(0, 0.01), title='Clipped sine mixture')
Spectrum(clipped_sine_mix).display(title='Spectrum of clipped sine mixture, with many additional harmonics')
Spectrum(sine_mix_3dBless).display(title='Spectrum of scaled down sine mixture, for comparison')

With all these building blocks, you now have plenty of options to creat new sounds through additive synthesis. A little trick that can reduce the amount of code is that `+` and `+=` can use NumPy arrays as their right hand operands too, so you don't need to turn the samples into `AudioSignal`s first before adding them to the the left hand operand.

#### Activity 7

Create new sounds following the principles of additive synthesis. You can first add multiple sines together and then apply an ADSR curve, or apply individual ADSR curves to the individual sines before summation to have additional control. Try to create at least one sound for each of the following types:

1. Harmonic sounds: Adding multiple sine waves with frequencies that are integer multiples of the same base frequency gives a sound that is perceived as having the same pitch as a single sine with that base frequency. Changing the amplitude of the sine components (known as _harmonics_) will influence the tonal _colour_ or _timbre_, which is another defining characteristic of a musical instrument. Note that not all multiples need to be present in the signal, which is equivalent to setting the corresponding amplitude to zero. You can leave gaps or alternate (e.g. using only even or odd harmonics), and even exclude the base frequency (or _fundamental frequency_). Have a look at the examples of a couple of musical instruments playing the same note below. Note for instance the characteristic dominance of odd harmonics in the clarinet signal.
2. Multi-pitch sounds: Adding a few sine waves with frequencies that form simple ratios (i.e. fractions with simple digits) generally leads to the perception of multiple, simultaneously played pitches. Some ratios lead to more pleasantly sounding (or _consonant_) combinations than others, but the more frequencies you add the higher the chance of _dissonance_. You can try to use some of the common musical intervals listed in the table below, ranked according to increasing dissonance, or just play around with simple fractions. Note that you can also use integer multiples of these fractions (thereby creating harmonics for each frequency component) to change the tonal colour of the result.
3. Inharmonic sounds: Adding several sines with arbitrary frequencies causes the perception of pitch to diminish. Instead a non-specific percussive sound will be heard. Try to combine sufficient sines such that you no longer can make out a tone.

Beware of the Nyquist frequency, which sets an upper limit to the frequencies you can create before aliasing occurs. Aliasing manifests itself as a reflection of the frequency, meaning that amplitudes in the spectrum can end up at a different frequency from where you intend them to be. This can add unexpected inharmonic components to harmonic signals, so check your spectra to verify.

In [ ]:
violin_signal = AudioSignal('../data/Violin.arco.ff.sulG.A4.stereo.wav')
violin_signal.play()
Spectrum(violin_signal).display(x_range=(0,5000), title='Spectrum of a violin playing an A4 note (440 Hz)')
clarinet_signal = AudioSignal('../data/EbClarinet.ff.A4.stereo.wav')
clarinet_signal.play()
Spectrum(clarinet_signal).display(x_range=(0,5000), title='Spectrum of a clarinet playing an A4 note (440 Hz)')
flute_signal = AudioSignal('../data/Flute.nonvib.ff.A4.stereo.wav')
flute_signal.play()
Spectrum(flute_signal).display(x_range=(0,5000), title='Spectrum of a flute playing an A4 note (440 Hz)')
trumpet_signal = AudioSignal('../data/Trumpet.novib.ff.A4.stereo.wav')
trumpet_signal.play()
Spectrum(trumpet_signal).display(x_range=(0,5000), title='Spectrum of a trumpet playing an A4 note (440 Hz)')

<table>
  <thead>
    <tr><th>Interval Name</th><th>Frequency Ratio</th></tr>
  </thead>
  <tbody>
    <tr><td>Unison</td><td>1/1</td></tr>
    <tr><td>Octave</td><td>2/1</td></tr>
    <tr><td>Perfect Fifth</td><td>3/2</td></tr>
    <tr><td>Perfect Fourth</td><td>4/3</td></tr>
    <tr><td>Major Third</td><td>5/4</td></tr>
    <tr><td>Major Sixth</td><td>5/3</td></tr>
    <tr><td>Minor Third</td><td>6/5</td></tr>
    <tr><td>Minor Sixth</td><td>8/5</td></tr>
    <tr><td>Tritone</td><td>7/5</td></tr>
    <tr><td>Minor Seventh</td><td>9/5</td></tr>
    <tr><td>Whole Tone</td><td>9/8</td></tr>
    <tr><td>Major Seventh</td><td>15/8</td></tr>
    <tr><td>Semitone</td><td>16/15</td></tr>
</table>

##### Harmonic sounds

In [ ]:
# your code here

##### Multi-pitch sounds

In [ ]:
# your code here

##### Inharmonic sounds

In [ ]:
# your code here

### 7. Subtractive Synthesis

Another way of creating sounds is subtractive synthesis, which is how most analogue synthesisers work. As the name implies, sound is shaped by starting from a complex waveform (having many harmonics) and selectively reducing frequency components. Triangular, square and sawtooth waves are all commonly used.

Modifying selective frequency components in a signal is known as filtering. Conceptually you can think of it as taking the spectrum of a sound, changing some frequencies and then recreating a sound from the modified spectrum. Such operations can be performed on a `Spectrum` using the `set_magnitude(value, start, end)` and `modify_magnitude(amount, start, end)` methods. They respectively overwrite magnitudes with `value` or modify the existing magnitudes by `amount` in the frequency band going from `start` to `end` Hz. Depending on whether the `Spectrum` was created with `dB=True` or `dB=False` in the constructor, the values need to be expressed in dB or as linear amplitudes. Passing a `Spectrum` variable to an `AudioSignal` constructor will then reconstruct the audio signal from its spectrum.

In [ ]:
square_signal = AudioSignal(signal.square(2*np.pi*freq*time), samplerate)
square_spectrum = Spectrum(square_signal)
mod_square_spectrum = square_spectrum.modify_magnitude(-50, 440, 880).modify_magnitude(-50, 5000, 10000)
square_signal.play()
square_spectrum.display(title='Original spectrum')
AudioSignal(mod_square_spectrum).play(normalize=True)
mod_square_spectrum.display(title='Modified spectrum')

In [ ]:
triangle_signal = AudioSignal(signal.sawtooth(2*np.pi*freq*time, width=0.5), samplerate)
triangle_spectrum = Spectrum(triangle_signal)
triangle_signal.play()
triangle_spectrum.display(title='Original spectrum')
mod_triangle_spectrum = triangle_spectrum.set_magnitude(-120, end=8000)
AudioSignal(mod_triangle_spectrum).play(normalize=True)
mod_triangle_spectrum.display(title='Modified spectrum')

Drastically removing frequencies like this is quite invasive and unnatural, which has as a consequence that the audio signal recreated from the modified spectrum can have amplitudes higher than 1, even though we only removed frequencies (Verify this by plotting the waveform). The proper way to solve this would be to sufficiently reduce the gain of the recreated waveform with `apply_gain`, such that the range of its samples falls within [-1, 1] again. However, in order to save the time required to figure out what the appropriate amount of gain reduction would be, we can take the shortcut of passing `normalize=True` to the `.play()` method as shown above. This won't change the samples itself, but will normalize them to have a peak amplitude of 1 just for playback. Beware that using this option makes it impossible to aurally distinguish between waveforms of different amplitudes, so only use it in the right context.

#### Activity 8

Use the `set_magnitude` and `modify_magnitude` methods to create new sounds from complex waveforms. See if you can apply each of the following four classic filter shapes:
1. Low-pass filter: all frequencies above a threshold are suppressed, everything below is left unchanged.
2. High-pass filter: all frequencies below a treshold are suppressed, everything above is left unchanged.
3. Band-pass filter: all frequencies in a certain frequency range are left unchanged, the rest is suppressed.
4. Band-stop filter: all frequencies in a certain frequency range are suppressed, the rest is unchanged

The frequency range that is left unchanges is also called the _pass-band_, whereas the frequency range that gets suppressed is called the _stop-band_. Note that `start` and `end` default to the lowest and highest frequency respectively, which will allows you to simplify your code.

##### Low-pass filter

In [ ]:
# your code here

##### High-pass filter

In [ ]:
# your code here

##### Band-pass filter

In [ ]:
# your code here

##### Band-stop filter

In [ ]:
# your code here

In practice, filtering is rarely done in the frequency domain, like we just did. It has a number of drawbacks such as relatively high computational requirements and limited real-time processing capabilities. Filtering in the time domain solves most of these problems, while introducing a couple new ones. The most important drawback of time-domain filtering is that the transitions between pass-bands and stop-bands can't be made as steep as in the frequency domain. The suppression cannot be as strong, and the transition needs to happen more gradually in a frequency range around the critical frequency (called transition band). This is not necessarily a significant problem for artistic applications like sound design, as the smooth transitions can sound pleasant, but is important for applications in telecommunications etc.

The width of this transition band, along with the steepness of the transition slope and the extent of unwelcome side-effects on the signal in the stop-band and pass-band are all factors that can be traded off against each other in order to find the right compromise for a specific application. Many different filter design algorithms exist, each making different trade-offs. Various options are available in the [`scipy.signal library`](https://docs.scipy.org/doc/scipy/reference/signal.html#filter-design).

We won't go into the details of choosing between different filter designs, but will experiment with the [`signal.firls(taps, band_freqs, band_amps, fs=samplerate)`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firls.html) function. You specify the desired filter characteristic as a list with edges of the frequency bands and the amplitudes at those points. Crucially, you describe pass bands and stop bands independently from each other, i.e. the end of a pass pand is not automatically the start of a stop band, but you need to leave room for a transition band between them. So instead of specifying a single cutoff frequency of 4000 Hz, you give a list `[0, 3800, 4200, f_Nyq]` as the second argument. The range between 3800 and 4200 Hz is then free for use as transition band. If you require a narrower transition band, you can bring the end of the lower band closer to the start of the upper band, such as `[0, 3900, 4100, f_Nyq]`. As the third parameter, you pass a list of desired amplitudes for the frequencies in `band_freqs`. So `[1, 1, 0, 0]` would create a low-pass filter with cutoff around 4000 Hz, while `[0, 0, 1, 1]` would create a high-pass filter with approximately that cutoff frequency. This notation is quite flexible and allows to specify multiple pass and stop bands at once, each with different amplitudes and transition bands. The first parameter `taps` needs to be odd and trades off quality for cost. A high number lets the actual filter approach the desired specification better, but comes at a higher cost (computation time and number of electronic components required when implementing in hardware, less relevant for software).

The filter parameters (or coefficients) returned from `signal.firls` are then passed into the `.filter(coeffs)` method of a `Spectrum` variable, returning the filtered spectrum. Do play around with the example below and see how the width of the transition band and the number of filter taps influence the filter output. For instance, can you see how narrowing the transition band creates frequency-dependent distortions (known as ripples) in the stop-band? It becomes more obvious when using less filter taps, because the "budget" in the complex trade-off procedure diminishes, meaning that it becomes impossible to satisfy all filter requirements (narrow transition band, absence of ripples, etc.) at once.

In [ ]:
@widgets.interact(
    transition_band=widgets.IntRangeSlider(min=100, max=10000, step=100, value=[3000, 5000], description='Transition band',
                                           style={'description_width': 'initial'}, layout=widgets.Layout(width='50%')),
    taps=widgets.IntSlider(min=1, max=101, step=2, value=33, description='Number of filter taps',
                           style={'description_width': 'initial'}, layout=widgets.Layout(width='50%')),
)
def low_pass(transition_band, taps):
    coeffs = signal.firls(taps, [0, transition_band[0], transition_band[1], samplerate/2], [1, 1, 0, 0], fs=samplerate)
    filtered_signal = square_signal.filter(coeffs)
    filtered_signal.play(normalize=True)
    Spectrum(filtered_signal).display(title='Spectrum of a square signal low-pass filtered in the time domain')

#### Activity 9

Filtering in the frequency domain has its own challenges regarding precise control of the frequency that forms the transition between bands. Can you think of why this might be?

_your answer here_

#### Activity 10

Create some other filter characteristics with `signal.firls` and play around with the parameters to see their influence (do ignore the interactive widget of the example and change parameters manually instead to keep your code simple). Especially band-pass and band-stop filters are interesting, because you now have _two_ stop-bands or pass bands, respectively, as an additional complication. Narrowing the distance between them (so the width of the single pass-band or stop-band) makes it even harder for the filter design algorithm to come up with a solution that satisfies all constraints.

In [ ]:
# your code here

### 10. Going further

If you've reached this point, congratulations, that's all you need to know for the remaining session. However, if you want to dig deeper into the subject of sound synthesis, now or after the sprint, there are a couple more things you can do to create more intricate sounds.

First, try combinining all techniques, e.g. adding an ADSR curve to a sound created by subtractive synthesis. Then you can chain multiple `AudioSignal`s in time with the `&` operator. For instance, combine multiple harmonic sounds of different frequencies to create a short melody or multiple percussive sounds to creat a beat.

Applying an ADSR curve is one specific way to modify the amplitude of an audio signal, but the principle can be applied more generally too. [_Amplitude modulation_](https://en.wikipedia.org/wiki/Amplitude_modulation) is achieved by modulating (i.e. pointwise multiplication of) one signal with another of equal length. It can be performed in code by applying the `*` operator to two `AudioSignal`s. For audio applications, it is most common to modulate a signal with a sine wave of a few Hertz (called a low-frequency oscillator or LFO on synths). Although the modulating sine wave is not audible in itself, its effect on the original signal is known as a [tremolo](https://en.wikipedia.org/wiki/Tremolo_(electronic_effect)).

The tremolo effect is part of what gives a [Leslie speaker](https://en.wikipedia.org/wiki/Leslie_speaker) its [characteristic sound](https://www.soundonsound.com/techniques/synthesizing-rest-hammond-organ-part-1). Since Leslie speakers were originally created to use with Hammond organs, which produce sound by [additive synthesis](https://www.soundonsound.com/techniques/synthesizing-tonewheel-organs-part-1), this combination would be a natural start for your experiments. In contrast, the metal rod on some electric guitars that is sometimes referred to as ["tremolo arm"](https://en.wikipedia.org/wiki/Vibrato_systems_for_guitar) actually has a misleading name, because it produces a vibrato (frequency modulation) effect. There are more musical applications of amplitude modulation than just tremolo though, as described in this [Synth Secrets article](https://www.soundonsound.com/techniques/amplitude-modulation), so do experiment with different waveforms and frequencies.

Do note, however, that recreating classical synth sounds using these basic building blocks will be quite challenging. The given building blocks are all created digitally, meaning mathematically exact, while physical constraints of the original analogue electronics meant that they were far from (mathematically) perfect. Therefore a big part of the skill of creating a digital synthesiser comes down to recreating the imperfections of analogue equipment that our ears have come to expect and like. The resulting implementations are far more complicated than a simple digital implementation of the underlying principle would be. Don't let that stop you having fun though!